In [3]:
import threading
import time

def num_print(num):
    for i in range(num):
        print("func call ", i)
        time.sleep(0.5)

th = threading.Thread(target=num_print,args=(5,))
th.start()

for i in range(5):
    print("main call ", i)
    time.sleep(0.5)

th.join()

func call main call  0
 0
main call  1
func call  1
main call  2
func call  2
main call  3
func call  3
main call  4
func call  4


### Subclassing threading.Thread, remember when the thread.start() is run the run() inside the thread will be run.

In [11]:
class Mythread(threading.Thread):
    def __init__(self, num):
        super().__init__()
        self.num = num

    def run(self):
        for i in range(self.num):
            print("func call ", i)
            time.sleep(0.5)

th = Mythread(5)
th.start()

for i in range(5):
    print("main call ", i)
    time.sleep(0.5)
th.join()

func call  0
main call  0
func call  1
main call  1
func call main call  2
 2
main call func call  3
 3
func call  4
main call  4


### Thread Synchronization
### When multiple threads access shared resources (e.g., a variable or a file), it can lead to data corruption or unexpected results. To prevent this, we use synchronization mechanisms like Lock or RLock.

In [ ]:
import threading

counter = 0
lock = threading.Lock()

def increment():
    global counter
    count = 0
    while counter < 1000:
        with lock:
            counter += 1    
        count += 1
        if counter == 1000:
            print('count : ',count)
            break
        time.sleep(0.01)

threads = [threading.Thread(target=increment) for _ in range(5)]

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

print("final counter : ", counter)

count :  200
final counter :  1000


In [30]:
import threading
import time

event = threading.Event()  # The stoplight (Event)

def worker():
    print("Worker: Waiting for the raw materials to arrive.")
    event.wait()  # Wait for the green light (event is set)
    print("Worker: Got the materials! Starting work now.")

# Start the worker thread
thread = threading.Thread(target=worker)
thread.start()

# Simulate the boss getting materials ready
time.sleep(3)
print("Boss: Raw materials are ready!")
event.set()  # Turn the light green (signal the worker)

thread.join()  # Wait for the worker to finish


Worker: Waiting for the raw materials to arrive.
Boss: Raw materials are ready!
Worker: Got the materials! Starting work now.


In [ ]:
semaphore = threading.Semaphore(3)

def access_resource(name):
    with semaphore:
        print(f"{name} is accessing the resource.")
        threading.Event().wait(1)  # Simulating work
        print(f"{name} is done.")

threads = [threading.Thread(target=access_resource, args=(f"Thread-{i}",)) for i in range(4)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

Thread-0 is accessing the resource.
Thread-1 is accessing the resource.
Thread-2 is accessing the resource.
Thread-0 is done.Thread-1 is done.
Thread-2 is done.
Thread-3 is accessing the resource.

Thread-3 is done.


## Thread Communication
### Threads can communicate using shared variables or queue.

In [37]:
import threading
import queue

q = queue.Queue()

def producer():
    for i in range(5):
        q.put(i)
        time.sleep(0.5)
        print(f"Produced: {i}")

def consumer():
    while not q.empty():
        item = q.get()
        time.sleep(0.75)
        print(f"Consumed: {item}")

producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

producer_thread.start()

consumer_thread.start()
producer_thread.join()
consumer_thread.join()


Produced: 0
Consumed: 0
Produced: 1
Consumed: 1Produced: 2

Produced: 3
Consumed: 2
Produced: 4
Consumed: 3
Consumed: 4


# Thread Pooling
### Using a thread pool is efficient when managing many threads.

In [8]:
from concurrent.futures import ThreadPoolExecutor
import time

def task(n):
    print(f"Processing {n}")
    time.sleep(2/(n+1))
    return n * 2

with ThreadPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(task, i) for i in range(10)]
    results = [future.result() for future in futures]

print("Results:", results)


Processing 0
Processing 1
Processing 2
Processing 3
Processing 4
Processing 5
Processing 6
Processing 7
Processing 8
Processing 9
Results: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


### here in above code, three threads are executed simultanously, it just made the thread creating easy, instead of thread.start here is executor.submit() and for results of that threads you get from executor.result().<br>
### Very interesting thing here is even though the thread conpletes the first the results will be in same serial of starting the thread.

### What Can You Profile in Threads?
Thread Execution Time: How long each thread takes to execute.<br>
CPU and Memory Usage: The resources consumed by threads.<br>
Thread State: Whether a thread is running, waiting, or blocked.<br>
Deadlocks: Detecting if threads are stuck waiting for resources held by each other.<br>
Concurrency Issues: Ensuring thread synchronization is working as intended.


In [1]:
import threading
import cProfile

def worker():
    for _ in range(1000000):
        _ = 1 + 1  # Simulate computation

def profile_threads():
    threads = [threading.Thread(target=worker) for _ in range(5)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

cProfile.run('profile_threads()')

         601 function calls (542 primitive calls) in 0.674 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      5/1    0.272    0.054    0.077    0.077 363893543.py:4(worker)
        1    0.000    0.000    0.650    0.650 363893543.py:8(profile_threads)
        1    0.000    0.000    0.650    0.650 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 _weakrefset.py:39(_remove)
        5    0.000    0.000    0.000    0.000 _weakrefset.py:85(add)
        2    0.000    0.000    0.000    0.000 base_events.py:734(time)
        1    0.000    0.000    0.000    0.000 events.py:86(_run)
        1    0.000    0.000    0.000    0.000 history.py:839(_writeout_output_cache)
        1    0.000    0.000    0.000    0.000 ioloop.py:742(_run_callback)
        1    0.000    0.000    0.000    0.000 iostream.py:616(_flush)
        1    0.000    0.000    0.000    0.000 iostream.py:710(_flush_buffers)
        1    0.000    0.000   

In [6]:
import threading
import yappi
import time

def worker():
    for _ in range(100):
            _ = 1 + 1  # Simulate computation


# Enable Yappi profiling
yappi.start()

threads = [threading.Thread(target=worker) for _ in range(3)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

# Stop profiling
yappi.stop()

# Print thread-level statistics
yappi.get_thread_stats().print_all()



name           id     tid              ttot      scnt        
Thread         7      32168            0.125000  2         
_MainThread    0      30404            0.062500  21        
Thread         6      30444            0.015625  2         
Thread         4      33732            0.015625  2         
Thread         12     26212            0.000000  1         
Thread         13     32264            0.000000  1         
Thread         1      28044            0.000000  2         
Thread         15     31468            0.000000  1         
Thread         2      33740            0.000000  2         
Thread         9      33632            0.000000  1         
..avingThread  5      17596            0.000000  3         
Thread         14     6784             0.000000  1         
Thread         11     30384            0.000000  1         
Thread         10     17748            0.000000  1         
Thread         17     33364            0.000000  1         
Thread         16     21648          

In [8]:
import threading
import faulthandler
import time

faulthandler.enable()  # Enable fault handler

lock = threading.Lock()

def worker1():
    with lock:
        time.sleep(5)  # Simulate a long operation

def worker2():
    with lock:
        print("This will wait for worker1 to release the lock.")

thread1 = threading.Thread(target=worker1)
thread2 = threading.Thread(target=worker2)

thread1.start()
thread2.start()

thread1.join()
thread2.join()


This will wait for worker1 to release the lock.
